Uploud the data

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import math
import statsmodels.api as sm

from random import*
from google.colab import drive
from scipy.stats import multivariate_normal
%matplotlib inline

from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split, GridSearchCV, KFold,RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA


# Step 2: Mount Google Drive
drive.mount('/content/drive')

# Step 4: Read the CSV file into a DataFrame
Xy_train = pd.read_csv("/content/drive/Shareddrives/ML/חלק ב/AirlineSatisfaction Dataset/Xy_train.csv")



Pre-Processing

In [ ]:
train_data = Xy_train

# Remove unwanted class
train_data["Class"][train_data["Class"] == "IT IS SO BORING WORKING IN AN AIRPORT'S DESK OH MY GODDDDD"] = None
train_data["Class"][train_data["Class"] == 'Unknown'] = None

for column in train_data[:-1].columns:train_data[column] = train_data[column].fillna(np.random.choice(train_data[column][~train_data[column].isna()]))

to_plot = train_data[["Type of Travel","Inflight wifi service" , "Cleanliness","Customer Type","Inflight service",'Flight Distance','satisfaction']].copy(deep=True)

categorical_columns = train_data.select_dtypes(include=['object', 'category']).columns
label_encoders = {}
for column in categorical_columns:
    le = LabelEncoder()
    train_data[column] = le.fit_transform(train_data[column])
    label_encoders[column] = le
cat = []
for column in train_data.columns:
  if len(train_data[column].unique()) <=30 and column != 'satisfaction':
    cat.append(column)
train_data = pd.get_dummies(train_data, columns=cat)

print('______________________________________________________________________________________________________________________-')
for column in train_data.columns: print(f"column : {column} -> {list(train_data[column].unique())}")

y_train = train_data['satisfaction']
X_train = train_data.drop(columns=['satisfaction'])

####  look at the data to see if there are seen clusters

In [ ]:
sns.pairplot(to_plot, hue='satisfaction')
plt.show()

 Since this data is four dimensional, we can reduce it to two which allows to visualize the data with low reduction in explained variance

In [ ]:

pca = PCA(n_components=2)

In [ ]:
pca.fit(X_train)


In [ ]:
train_data_pca = pca.transform(X_train)
train_data_pca = pd.DataFrame(train_data_pca, columns=['PC1', 'PC2'])
train_data_pca['satisfaction'] = y_train

In [ ]:
sns.scatterplot(x='PC1', y='PC2', hue='satisfaction', data=train_data_pca)
plt.show()

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2, max_iter=300, n_init=10, random_state=42)

In [ ]:
kmeans.fit(X_train)

scores

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import adjusted_rand_score
import matplotlib.pyplot as plt

# 4. הרצת K-Means עם 2 אשכולות
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(X_train)
cluster_labels = kmeans.labels_
k_predict = kmeans.predict(X_train)

ari_score = adjusted_rand_score(y_train.apply(lambda x :not x), cluster_labels)
print(f'Adjusted Rand Index (ARI): {ari_score}')
f1 = f1_score(y_train.apply(lambda x :not x), k_predict)
print(f'F1 Score: {f1}')

In [ ]:
train_data_pca['satisfaction'] = k_predict

#### present the points with the cluster centers

In [ ]:
sns.scatterplot(x='PC1', y='PC2', hue='satisfaction', data=train_data_pca, palette='Accent')
plt.scatter(pca.transform(kmeans.cluster_centers_)[:, 0], pca.transform(kmeans.cluster_centers_)[:, 1], marker='+', s=100 ,color='red')
plt.show()

#### Now, what can we do if we dont know the number of clusters ?
#### what are the main two clusters properies we want?
Homogeneity, seperation

In [ ]:
from sklearn.metrics import silhouette_score, davies_bouldin_score
from tqdm import tqdm
iner_list = []
dbi_list = []
sil_list = []

for n_clusters in tqdm(range(2, 10, 1)):
    kmeans = KMeans(n_clusters=n_clusters, max_iter=300, n_init=10, random_state=42)
    kmeans.fit(X_train)
    assignment = kmeans.predict(X_train)

    iner = kmeans.inertia_
    sil = silhouette_score(X_train, assignment)
    dbi = davies_bouldin_score(X_train, assignment)

    dbi_list.append(dbi)
    sil_list.append(sil)
    iner_list.append(iner)

plot measure

In [ ]:
plt.plot(range(2, 10, 1), iner_list, marker='o')
plt.title("Inertia")
plt.xlabel("Number of clusters")
plt.show()

plt.plot(range(2, 10, 1), sil_list, marker='o')
plt.title("Silhouette")
plt.xlabel("Number of clusters")
plt.show()

plt.plot(range(2, 10, 1), dbi_list, marker='o')
plt.title("Davies-Bouldin")
plt.xlabel("Number of clusters")
plt.show()


Inertia is a measure of homogenity - Sum of squared distances of samples to their closest cluster center.

Davies bouldin score and Silhouette measure mixed homogenity and separation.

Better clustring represents by lower inertia and Davies-Bouldin measures with higher Silhouette score


In [ ]:
from sklearn.cluster import KMeans
n_clusters = 5
kmeans = KMeans(n_clusters=n_clusters, max_iter=300, n_init=10, random_state=42)
kmeans.fit(X_train)

kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(X_train)
cluster_labels = kmeans.labels_
train_data_pca['satisfaction'] = kmeans.predict(X_train)

# ari_score = adjusted_rand_score(y_train.apply(lambda x :not x), cluster_labels)
# print(f'Adjusted Rand Index (ARI): {ari_score}')
# f1 = f1_score(y_train.apply(lambda x :not x), k_predict)
# print(f'F1 Score: {f1}')

sns.scatterplot(x='PC1', y='PC2', hue='satisfaction', data=train_data_pca, palette='Accent')
plt.scatter(pca.transform(kmeans.cluster_centers_)[:, 0], pca.transform(kmeans.cluster_centers_)[:, 1], marker='+', s=100 ,color='red')
plt.show()